In [11]:
import requests
import pandas as pd
import os
from datetime import datetime
from io import StringIO
import json

In [ ]:
def get_list(url:str, types:str=None):
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    if types=="protocol":
        df["protocol"] = df["name"].apply(lambda x: x.lower().replace(" ", "-"))
    df.to_excel(f"{types}.xlsx", index=False)
    print(df.shape, df.columns)
    return df

In [4]:
def request_csv(url:str):
    response = requests.get(url, allow_redirects=True)
    data = response.content.decode("utf-8")
    if data == "Internal server error":
        return None
    return pd.read_csv(StringIO(data), sep=",")

# Download Protocol

In [ ]:
url = "https://api.llama.fi/protocols"
df_protocol = get_list(url, "protocol")
df_protocol.head()

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

df_protocol = pd.read_excel("protocol.xlsx")

failed = []
os.makedirs("df_protocol", exist_ok=True)

for v in df_protocol["protocol"]:
    url = "https://api.llama.fi/dataset/%s.csv" % v
    df = request_csv(url)
    if df is None or df.empty:
        failed.append(name)
        continue

    try:
        df = df.reset_index()
        total = df.iloc[1].astype(str).str.contains("Total")
        tvl = df.iloc[2].astype(str).str.contains("TVL")
        col = df.columns[total & tvl]

        if len(col) == 0:
            print(v, "Columns not found")
            # failed.append(v)
            continue
        if len(col) > 1:
            print(v, "Multiple columns found")
            print(col)
            # failed.append(v)
            continue

        df = df.iloc[4:]
        df["Timestamp"] = df["Timestamp"].apply(lambda x: datetime.fromtimestamp(x))
        col = col.item()    
        df = df[["Date", "Timestamp", col]]
        df.to_csv(f"df_protocol/{v}.csv", index=False)
    except Exception as e:
        print(v, e)

failed

# Download Chain

In [5]:
url = "https://api.llama.fi/v2/chains"
df_chain = get_list(url, "chain")
df_chain.head()

(298, 6) Index(['gecko_id', 'tvl', 'tokenSymbol', 'cmcId', 'name', 'chainId'], dtype='object')


,gecko_id,tvl,tokenSymbol,cmcId,name,chainId
0,harmony,1.645639e+06,ONE,3945,Harmony,1666600000
1,mantle,4.591852e+08,MNT,27075,Mantle,5000
2,binancecoin,4.226293e+09,BNB,1839,BSC,56
3,aurora-near,1.615444e+07,AURORA,14803,Aurora,1313161554
4,x-layer,9.810801e+06,None,None,X Layer,None


In [ ]:
failed = []
os.makedirs("df_chain", exist_ok=True)

for name in df_chain["name"]:
    name = name.replace(" ", "_")
    try:
        url = "https://api.llama.fi/simpleChainDataset/%s?" % name
        df = request_csv(url)
        if df is None or df.empty:
            failed.append(name)
            continue

        df = df.head(1).T.iloc[1:].reset_index()
        df["date"] = df["index"].map(lambda x: datetime.strptime(x, '%d/%m/%Y'))
        df["date"] = df["date"].dt.strftime('%Y-%m-%d')
        df[name] = df[0]
        df = df[["date", name]]
        df.to_csv(f"df_chain/{name}.csv", index=False)
        
    except Exception as e:
        print(name, e)
        print(data)

# Remove Zero TVL data

In [ ]:
for v in os.listdir("df_protocol"):
    df = pd.read_csv(f"df_protocol/{v}")
    df = df.iloc[:, 2:].fillna(0)
    try:
        if df.sum().sum() == 0:
            os.remove(f"df_protocol/{v}")
    except Exception as e:
        print(v, e)

for v in os.listdir("df_chain"):
    df = pd.read_csv(f"df_chain/{v}")
    df = df.iloc[:, 2:].fillna(0)
    try:
        if df.sum().sum() == 0:
            os.remove(f"df_chain/{v}")
    except Exception as e:
        print(v, e)

In [ ]:
len(os.listdir("df_chain"))

297

In [ ]:
len(os.listdir("df_protocol"))

4643